In [2]:
from siphon.catalog import TDSCatalog, Dataset
import metpy
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh
import pyart
import matplotlib as mpl
import pytz
from datetime import timezone
from natsort import natsorted
import sys
import gc
import warnings
warnings.filterwarnings('ignore');

#Get the list of current files on the THREDDS server
#Note that the TDSCatalog takes the url of the folder you want the files from as an argument
cat = TDSCatalog('https://thredds.ucar.edu/thredds/catalog/grib/NCEP/MRMS/BaseRef/catalog.xml')
ds = cat.datasets[list(cat.datasets)[-1]].remote_access(use_xarray=True).metpy.parse_cf()

min_lon = -80+360
min_lat = 35
max_lon = -65+360
max_lat = 50

mask_lon = (ds.lon >= min_lon) & (ds.lon <= max_lon)
mask_lat = (ds.lat >= min_lat) & (ds.lat <= max_lat)
cropped_ds = ds.where(mask_lon & mask_lat, drop=True)
del ds

dirpath = '/home/metweb/public_html/map_room/mrms_pics/'
import time
import metpy.calc as mpcalc
import metpy.plots as mpplots
# Stuff for plotting the NWS Radar data in correct colormap and levels
cmap = "pyart_HomeyerRainbow"
levs = np.linspace(0, 80, 41, endpoint=True)
ticks = np.linspace(0, 80, 9, endpoint=True)
# normalize the colormap based on the levels provided above
norm = mpl.colors.BoundaryNorm(levs, 256)
sub = 'BaseReflectivity'
ref = cropped_ds[next((s for s in list(cropped_ds.variables) if sub in s), None)]
print(len(ref))
count = 0
for i in range(0,len(ref),2):
    if( count < 100000):
        start_time = time.time()
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
        ax.set_extent([-80, -65, 35, 50], crs=ccrs.PlateCarree())
        pc = ax.pcolormesh(ref.lon,ref.lat,ref[i,0,:,:],cmap=cmap, norm=norm, transform=ccrs.PlateCarree())
        ax.add_feature(cfeature.STATES.with_scale('10m'))
        props = dict(boxstyle='round', facecolor='white', alpha=1.0)
        ax.text(0.01, 0.01, pytz.utc.localize(pd.to_datetime(ref.time[i].values)).astimezone(pytz.timezone("US/Eastern")).strftime("%m-%d-%Y %I:%M %p"), transform=ax.transAxes, fontsize=14,verticalalignment='bottom', bbox=props,zorder=100)

        metar_cat = TDSCatalog('https://thredds.ucar.edu/thredds/catalog/nws/metar/ncdecoded/catalog.xml')
        #metar_file = metar_cat.datasets.filter_time_nearest(ref.time[i].values).remote_open()
        metar_dataset = metar_cat.datasets['Feature Collection']
        ncss = metar_dataset.subset()
        from datetime import datetime
        ncss.variables
        query = ncss.query()
        query.lonlat_box(north=50, south=35, east=-65, west=-80)
        query.time(pd.to_datetime(ref.time[i].values))
        query.variables('air_temperature', 'dew_point_temperature',
                        'wind_speed', 'wind_from_direction', 'cloud_area_fraction')
        query.accept('csv')
        data = ncss.get_data(query)
        from metpy.units import units
        # Since we used the CSV data, this is just a dictionary of arrays
        lats = data['latitude']
        lons = data['longitude']
        tair = (data['air_temperature'] * (9/5) ) + 32
        dewp = (data['dew_point_temperature'] * (9/5) ) + 32



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



ModuleNotFoundError: No module named 'natsort'